In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix



In [2]:
import os
path = os.getcwd() + '/ml1m/'

In [3]:
ratings_df = pd.read_csv(f'{path}preprocessed/ratings.txt', sep='\t')
ratings_df

,uid,pid,rating,timestamp
0,0,872,5,978300760
1,0,537,3,978302109
2,0,679,3,978301968
3,0,2606,4,978300275
4,0,1790,5,978824291
...,...,...,...,...
940958,6039,810,1,956716541
940959,6039,813,5,956704887
940960,6039,477,5,956704746
940961,6039,815,4,956715648


In [4]:
movies_df = pd.read_csv(f'{path}movies.dat',  sep="::", names=["movie_id", "movie_name", "genre"], header=None, encoding='latin-1')
movies_df

/var/folders/8j/vb8qq8v168531_7qk_cwrfyc0000gn/T/ipykernel_15835/736248507.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies_df = pd.read_csv(f'{path}movies.dat',  sep="::", names=["movie_id", "movie_name", "genre"], header=None, encoding='latin-1')


,movie_id,movie_name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [5]:
mapping_df = pd.read_csv(f'{path}preprocessed/products.txt', sep='\t', index_col='new_id')
mapping_df

,raw_dataset_id
new_id,
0,2
1,3
2,4
3,5
4,6
...,...
3025,3947
3026,3948
3027,3949


In [6]:
# Load the data
movies = pd.merge(movies_df, mapping_df, left_on='movie_id', right_on='raw_dataset_id')
movies

,movie_id,movie_name,genre,raw_dataset_id
0,2,Jumanji (1995),Adventure|Children's|Fantasy,2
1,3,Grumpier Old Men (1995),Comedy|Romance,3
2,4,Waiting to Exhale (1995),Comedy|Drama,4
3,5,Father of the Bride Part II (1995),Comedy,5
4,6,Heat (1995),Action|Crime|Thriller,6
...,...,...,...,...
3025,3947,Get Carter (1971),Thriller,3947
3026,3948,Meet the Parents (2000),Comedy,3948
3027,3949,Requiem for a Dream (2000),Drama,3949
3028,3950,Tigerland (2000),Drama,3950


In [7]:
movies['id'] = range(len(movies))
movies.set_index('id', inplace=True)
movies = movies.drop(columns=['movie_id', 'raw_dataset_id'])

In [8]:
movies

,movie_name,genre
id,,
0,Jumanji (1995),Adventure|Children's|Fantasy
1,Grumpier Old Men (1995),Comedy|Romance
2,Waiting to Exhale (1995),Comedy|Drama
3,Father of the Bride Part II (1995),Comedy
4,Heat (1995),Action|Crime|Thriller
...,...,...
3025,Get Carter (1971),Thriller
3026,Meet the Parents (2000),Comedy
3027,Requiem for a Dream (2000),Drama


In [9]:
data = pd.merge(ratings_df, movies, left_on='pid', right_on='id')

In [10]:
data

,uid,pid,rating,timestamp,movie_name,genre
0,0,872,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,872,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,11,872,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,14,872,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,16,872,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
940958,2908,2634,5,971971272,Born American (1986),Action|Drama|Thriller
940959,3313,2634,3,967947206,Born American (1986),Action|Drama|Thriller
940960,3617,2634,3,966598532,Born American (1986),Action|Drama|Thriller
940961,5678,2634,4,958619701,Born American (1986),Action|Drama|Thriller


In [11]:
user_movie_matrix = data.pivot_table(index='uid', columns='movie_name', values='rating').fillna(0)
user_movie_matrix

movie_name,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",...,Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zeus and Roxanne (1997),eXistenZ (1999)
uid,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,3.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,...,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0
6036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
user_movie_matrix_norm = user_movie_matrix.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1).clip(0)
user_movie_matrix_norm

movie_name,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",...,Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zeus and Roxanne (1997),eXistenZ (1999)
uid,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.987195,0.787195,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,0.421452,0.0,0.0,0.0,0.221452,0.621452,0.0,0.000000,0.021452,...,0.0,0.621452,0.000000,0.000000,0.0,0.421452,0.0,0.0,0.0,0.221452
6036,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.753201,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
6037,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000


In [13]:
sparse_matrix = csr_matrix(user_movie_matrix.values)
sparse_matrix

<6040x3030 sparse matrix of type '<class 'numpy.float64'>'
	with 940963 stored elements in Compressed Sparse Row format>

In [14]:
similarities = cosine_similarity(sparse_matrix)
similarities

array([[1.        , 0.08854972, 0.12862658, ..., 0.        , 0.19404654,
        0.12812071],
       [0.08854972, 1.        , 0.15904176, ..., 0.07030104, 0.07271236,
        0.21023644],
       [0.12862658, 0.15904176, 1.        , ..., 0.12556651, 0.10174285,
        0.14523598],
       ...,
       [0.        , 0.07030104, 0.12556651, ..., 1.        , 0.17599525,
        0.09171212],
       [0.19404654, 0.07271236, 0.10174285, ..., 0.17599525, 1.        ,
        0.23228967],
       [0.12812071, 0.21023644, 0.14523598, ..., 0.09171212, 0.23228967,
        1.        ]])

In [15]:
collab_filtering = pd.DataFrame(similarities, index=user_movie_matrix.index, columns=user_movie_matrix.index)
collab_filtering

uid,0,1,2,3,4,5,6,7,8,9,...,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039
uid,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.088550,0.128627,0.142031,0.085265,0.169181,0.063152,0.121438,0.207163,0.256404,...,0.183018,0.071553,0.073945,0.036611,0.098590,0.182278,0.133868,0.000000,0.194047,0.128121
1,0.088550,1.000000,0.159042,0.161169,0.115130,0.105031,0.306841,0.213239,0.201583,0.231698,...,0.091274,0.095773,0.270709,0.015296,0.191628,0.232495,0.201219,0.070301,0.072712,0.210236
2,0.128627,0.159042,1.000000,0.156805,0.065532,0.076303,0.141549,0.082354,0.131642,0.222612,...,0.096440,0.129792,0.165430,0.000000,0.103992,0.151918,0.112431,0.125567,0.101743,0.145236
3,0.142031,0.161169,0.156805,1.000000,0.033737,0.013912,0.108557,0.093812,0.098024,0.126487,...,0.140672,0.096469,0.394243,0.000000,0.071003,0.173513,0.133736,0.066057,0.069839,0.141763
4,0.085265,0.115130,0.065532,0.033737,1.000000,0.049022,0.122405,0.222524,0.265724,0.119207,...,0.095963,0.037222,0.063950,0.057526,0.171948,0.296565,0.173773,0.016505,0.030058,0.252232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.182278,0.232495,0.151918,0.173513,0.296565,0.098420,0.123811,0.229761,0.241970,0.344766,...,0.128264,0.218954,0.193680,0.111849,0.259604,1.000000,0.349315,0.120957,0.215416,0.421152
6036,0.133868,0.201219,0.112431,0.133736,0.173773,0.068085,0.115628,0.139411,0.212916,0.245526,...,0.149389,0.288134,0.134724,0.126367,0.135438,0.349315,1.000000,0.051797,0.263660,0.443741
6037,0.000000,0.070301,0.125567,0.066057,0.016505,0.068062,0.000000,0.020666,0.098539,0.120066,...,0.114346,0.111634,0.042207,0.000000,0.069229,0.120957,0.051797,1.000000,0.175995,0.091712


In [16]:
movie_features = movies.genre.str.get_dummies()
movie_features

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,
0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3026,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3027,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [17]:
movie_similarities = cosine_similarity(movie_features)
movie_similarities


array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.5       , 1.        , ..., 0.70710678, 0.70710678,
        0.70710678],
       ...,
       [0.        , 0.        , 0.70710678, ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.70710678, ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.70710678, ..., 1.        , 1.        ,
        1.        ]])

In [18]:
content_based = pd.DataFrame(movie_similarities, index=movies['movie_name'], columns=movies['movie_name'])
content_based

movie_name,Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),GoldenEye (1995),"American President, The (1995)",Dracula: Dead and Loving It (1995),...,Sorority House Massacre II (1990),Bamboozled (2000),Bootmen (2000),Digimon: The Movie (2000),Get Carter (2000),Get Carter (1971),Meet the Parents (2000),Requiem for a Dream (2000),Tigerland (2000),Two Family House (2000)
movie_name,,,,,,,,,,,,,,,,,,,,,
Jumanji (1995),1.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.816497,0.333333,0.000000,0.000000,...,0.0,0.000000,0.000000,0.666667,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
Grumpier Old Men (1995),0.0,1.000000,0.500000,0.707107,0.00000,1.000000,0.000000,0.000000,0.816497,0.500000,...,0.0,0.707107,0.500000,0.000000,0.000000,0.00000,0.707107,0.000000,0.000000,0.000000
Waiting to Exhale (1995),0.0,0.500000,1.000000,0.707107,0.00000,0.500000,0.000000,0.000000,0.816497,0.500000,...,0.0,0.707107,1.000000,0.000000,0.408248,0.00000,0.707107,0.707107,0.707107,0.707107
Father of the Bride Part II (1995),0.0,0.707107,0.707107,1.000000,0.00000,0.707107,0.000000,0.000000,0.577350,0.707107,...,0.0,1.000000,0.707107,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000
Heat (1995),0.0,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.666667,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.666667,0.57735,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Get Carter (1971),0.0,0.000000,0.000000,0.000000,0.57735,0.000000,0.000000,0.577350,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.577350,1.00000,0.000000,0.000000,0.000000,0.000000
Meet the Parents (2000),0.0,0.707107,0.707107,1.000000,0.00000,0.707107,0.000000,0.000000,0.577350,0.707107,...,0.0,1.000000,0.707107,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000
Requiem for a Dream (2000),0.0,0.000000,0.707107,0.000000,0.00000,0.000000,0.000000,0.000000,0.577350,0.000000,...,0.0,0.000000,0.707107,0.000000,0.577350,0.00000,0.000000,1.000000,1.000000,1.000000


In [31]:
user_id = 534
# Get top 5 similar users
similar_users = collab_filtering[user_id].sort_values(ascending=False)[1:6].index.tolist()
similar_users

[4066, 3874, 2738, 1289, 2547]

In [32]:
# Get movies rated highly by similar users
similar_user_movies = user_movie_matrix.loc[similar_users].mean(axis=0)
recommended_movies = similar_user_movies[similar_user_movies > 4.0].index.tolist()
recommended_movies

['Ghostbusters (1984)', 'Out of Africa (1985)', 'Sixth Sense, The (1999)']

In [33]:
user_movies = user_movie_matrix.loc[user_id]
similar_movies = content_based[user_movies > 0].mean(axis=0).sort_values(ascending=False)[:5].index.tolist()
similar_movies

/var/folders/8j/vb8qq8v168531_7qk_cwrfyc0000gn/T/ipykernel_15835/2650166328.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  similar_movies = content_based[user_movies > 0].mean(axis=0).sort_values(ascending=False)[:5].index.tolist()


['Hercules (1997)',
 'Stand by Me (1986)',
 'Wizard of Oz, The (1939)',
 'Muppet Treasure Island (1996)',
 'Mighty Joe Young (1998)']

In [34]:
recommendations = set(recommended_movies) | set(similar_movies)
recommendations

{'Ghostbusters (1984)',
 'Hercules (1997)',
 'Mighty Joe Young (1998)',
 'Muppet Treasure Island (1996)',
 'Out of Africa (1985)',
 'Sixth Sense, The (1999)',
 'Stand by Me (1986)',
 'Wizard of Oz, The (1939)'}

In [29]:
# Combine recommendations from collaborative and content-based filtering
movies[movies.iloc[:].isin(recommendations)]['movie_name'].tolist()

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [30]:
# # Hybrid recommender system
# def hybrid_recommender(user_id):
#     # Get top 5 similar users
#     similar_users = collab_filtering[user_id].sort_values(ascending=False)[1:6].index.tolist()
    
#     # Get movies rated highly by similar users
#     similar_user_movies = user_movie_matrix.loc[similar_users].mean(axis=0)
#     recommended_movies = similar_user_movies[similar_user_movies > 4.0].index.tolist()
    
#     # Get top 5 similar movies
#     user_movies = user_movie_matrix.loc[user_id]
#     similar_movies = content_based[user_movies > 0].mean(axis=0).sort_values(ascending=False)[:5].index.tolist()
    
#     # Combine recommendations from collaborative and content-based filtering
#     recommendations = set(recommended_movies) | set(similar_movies)
#     return movies[movies.iloc[:].isin(recommendations)]['movie_name'].tolist()


In [41]:
hybrid_recommender(1)

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'Braveheart (1995)',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
